__Diet Problem__

https://web.mit.edu/15.053/www/Excel_Solver.pdf

In [1]:
import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, LpAffineExpression
from pulp import LpBinary, LpMinimize, LpMaximize, lpSum, value

In [2]:
df = pd.DataFrame(
    [
        ["Brownie", 400, 3, 2, 2, 0.50],
        ["Ice Cream", 200, 2, 2, 4, 0.20],
        ["Cola", 150, 0, 4, 1, 0.30],
        ["Cheesecake", 500, 0, 4, 5, 0.80],
    ],
    columns=["food", "calories", "chocolate", "sugar", "fat", "cost"],
)

df

,food,calories,chocolate,sugar,fat,cost
0,Brownie,400,3,2,2,0.5
1,Ice Cream,200,2,2,4,0.2
2,Cola,150,0,4,1,0.3
3,Cheesecake,500,0,4,5,0.8


In [3]:
calories = dict(zip(df.food, df.calories))
chocolate = dict(zip(df.food, df.chocolate))
sugar = dict(zip(df.food, df.sugar))
fat = dict(zip(df.food, df.fat))
cost = dict(zip(df.food, df.cost))

food_vars = LpVariable.dict("food_vars", df.food.values, lowBound=0, cat="Integer")

In [4]:
model = LpProblem("Diet", LpMinimize)

# Objective function
model += lpSum([cost[food] * food_vars[food] for food in df.food])

# Constraints
model += (
    lpSum([calories[food] * food_vars[food] for food in df.food]) >= 500,
    "calories",
)
model += (
    lpSum([chocolate[food] * food_vars[food] for food in df.food]) >= 6,
    "chocolate",
)
model += (
    lpSum([sugar[food] * food_vars[food] for food in df.food]) >= 10,
    "sugar",
)
model += lpSum([fat[food] * food_vars[food] for food in df.food]) >= 8, "fat"

In [5]:
results = model.solve()

LpStatus[model.status]

'Optimal'

In [6]:
output = df[["food", "calories", "cost"]].copy()
output["should_eat"] = [food_vars[food].value() for food in df.food]

output

,food,calories,cost,should_eat
0,Brownie,400,0.5,0.0
1,Ice Cream,200,0.2,3.0
2,Cola,150,0.3,1.0
3,Cheesecake,500,0.8,0.0


In [7]:
(output.should_eat * output.calories).sum()

750.0

In [8]:
(output.should_eat * output.cost).sum().round(2)

0.9